# Suivi de l'apprentissage dans KERAS: illustration de TensorBoard pour suivre en temps réel l'apprentissage de modèles 

Dans ce notebook nous allons illustrer l'utilisation de **TensorBoard** pour suivre en temps réel l'apprentissage de modèles avec Keras.

Pour cela nous travaillerons sur le même jeu de données que celui du TP principal, le jeu MNIST, et nous utiliserons un modèle de régression multinomiale.

Le plan sera le suivant : 
* nous commencerons par mettre en forme le jeu de données et reproduire l'expérience du TP précédent
* nous verrons ensuite comment introduire un callback tensorboard et comment visualiser les résultats obtenus
* nous illustrerons comment cela peut-être utile pour mesurer l'impact du "learning rate" de l'algorithme de descente de gradient "classique"
* enfin, nous généraliserons ceci pour mesurer l'impact de l'algorithme d'optimisation, en considérant l'algorithme "Adam"


**Précision importante** : ce notebook n'est valable que pour **tensorflow v2**. La plupart des commandes devraient fonctionner pour une version plus ancienne de keras / tensorflow, mais les imports seront à revoir...

# 1) Mise en forme du jeu de données

## On commence par importer les packages & fonctions nécessaires

In [1]:
# import packages
import csv
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.callbacks import TensorBoard

# set random seed
np.random.seed(123)

In [2]:
%load_ext tensorboard

## On met ensuite en forme le jeu de données.

Pour cela : 
* on le charge via la fonction disponible dans Keras
* on ramène l'intensité des images dans la gamme \[0,1\]
* on convertit les images 28x28 en vecteurs (de longueur 84).
* on transforme les labels au format "one-hot-encoding"

In [3]:
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print('size of training data: ')
print(X_train.shape)
print('size of test data: ')
print(X_test.shape)

# convert our data type to float32 and normalize our data values to the range [0, 1]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# cast to vector
input_dim = 28*28 
X_train_vec = X_train.reshape(60000, input_dim) 
X_test_vec  = X_test.reshape(10000, input_dim)

# cast outcome to one-hot-encoding
Y_train = to_categorical(y_train)
Y_test  = to_categorical(y_test)

size of training data: 
(60000, 28, 28)
size of test data: 
(10000, 28, 28)


 On sépare enfin les données d'apprentissage en données d'apprentissage et de validation

In [4]:
from sklearn.model_selection import train_test_split
X_train_vec_train, X_train_vec_val, Y_train_train, Y_train_val = train_test_split(X_train_vec, Y_train, test_size=0.2, random_state=42)

## On définit un modèle de régression multinomiale

In [5]:
mlp = Sequential()
mlp.add(Dense(10, activation='softmax',input_dim=input_dim))

## On apprend notre modèle avec un algorithme SGD "standard", et ses paramètres par défaut.

In [6]:
# compile with standard sgd
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics='accuracy')
# fit
mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), batch_size=32, epochs=10, verbose=1) 

Epoch 1/10
1500/1500 [==============================] - 1s 767us/step - loss: 0.2600 - accuracy: 0.5067 - val_loss: 0.2010 - val_accuracy: 0.7268
Epoch 2/10
1500/1500 [==============================] - 1s 607us/step - loss: 0.1706 - accuracy: 0.7737 - val_loss: 0.1479 - val_accuracy: 0.8007
Epoch 3/10
1500/1500 [==============================] - 1s 613us/step - loss: 0.1344 - accuracy: 0.8175 - val_loss: 0.1234 - val_accuracy: 0.8264
Epoch 4/10
1500/1500 [==============================] - 1s 635us/step - loss: 0.1160 - accuracy: 0.8369 - val_loss: 0.1095 - val_accuracy: 0.8407
Epoch 5/10
1500/1500 [==============================] - 1s 644us/step - loss: 0.1048 - accuracy: 0.8473 - val_loss: 0.1005 - val_accuracy: 0.8504
Epoch 6/10
1500/1500 [==============================] - 1s 627us/step - loss: 0.0973 - accuracy: 0.8554 - val_loss: 0.0941 - val_accuracy: 0.8564
Epoch 7/10
1500/1500 [==============================] - 1s 627us/step - loss: 0.0918 - accuracy: 0.8607 - val_loss: 0.0894 -

# 2) Introduction d'un "callback" Tensorboard

## On apprend notre modèle en introduisant un "callback" Tensorboard

Pour cela, il suffit d'appeler la fonction Keras correspondante, et de spécifier un chemin vers un dossier.

In [7]:
# create model
mlp = Sequential()
mlp.add(Dense(10, activation='softmax',input_dim=input_dim))
# compile with standard sgd
mlp.compile(optimizer='sgd', loss='binary_crossentropy', metrics='accuracy')

# define tensorboard call-back
callback_tensorboard = TensorBoard(log_dir = "./Illustration-tensorboard/mlp_sgd-default", histogram_freq = 1)
# create output dir
if not os.path.exists("./Illustration-tensorboard"):
    os.mkdir("./Illustration-tensorboard")

# fit
mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), 
        callbacks = callback_tensorboard, batch_size=32, epochs=10, verbose=1) 

Epoch 1/10
   1/1500 [..............................] - ETA: 0s - loss: 0.3325 - accuracy: 0.0312WARNING:tensorflow:From C:\Users\mahep\.conda\envs\DeepLearning\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2516 - accuracy: 0.5495 - val_loss: 0.1974 - val_accuracy: 0.7371
Epoch 2/10
1500/1500 [==============================] - 1s 908us/step - loss: 0.1687 - accuracy: 0.7756 - val_loss: 0.1471 - val_accuracy: 0.7967
Epoch 3/10
1500/1500 [==============================] - 1s 900us/step - loss: 0.1340 - accuracy: 0.8166 - val_loss: 0.1234 - val_accuracy: 0.8239
Epoch 4/10
1500/1500 [==============================] - 1s 850us/step - loss: 0.1161 - accuracy: 0.8368 - val_loss: 0.1097 - val_accuracy: 0.8393
Epoch 5/10
1500/1500 [=============

## Une fois qu'on a lancé l'apprentissage, on peut suivre l'évolution des performances (en temps réel ou a posteriori) en lançant l'outil TensorBoard.

Cela peut se faire directement au sein du notebook via la commande ci-dessous, mais il est également possible de lancer cette commande dans un terminal. Tensorboard vous renverra alors une URL à ouvrir dans un navigateur internet. 

Cette deuxième solution est souvent la plus utile, car elle permet de suivre l'apprentissage d'un modèle lancé via un script python classique (et non au sein d'un notebook comme ici), et même de suivre dans une même interface plusieurs apprentissages lancés en parallèle.

**A noter que la commande tensorboard prend un dossier en argument**. Il s'agit du dossier dans lequel sont écrites les sorties des différents "callbacks" que l'on veut suivre. Chaque modèle que vous souhaitez suivre dans cette instance tensorboard doit écrire dans son propre sous-dossier (comme ci-dessus, dans le dosser *./Illustration-tensorboard/mlp_sgd-default*)


In [8]:
%tensorboard --logdir ./Illustration-tensorboard

Reusing TensorBoard on port 6006 (pid 13280), started 0:12:19 ago. (Use '!kill 13280' to kill it.)

# 3) Apprentissage de plusieurs modèles et comparaison via Tensorboard

## On apprend maintenant plusieurs modèles, en faisant varier le paramètre de "learning rate" et en introduisant un callback tensorboard à chaque fois.

Comme mentionné précédemment, il faut prendre soin de choisir un nouveau sous-dossier au sein du dossier "suivi" par Tensorboard pour chaque  modèle que l'on souhaite suivre. 

In [9]:
##################################
# assess effect of learning rate #
##################################
learning_rates = [0.0001,0.001,0.01,0.1,0.5]
from tensorflow.keras.optimizers import SGD
for eta in learning_rates:
    # create model
    mlp = Sequential()
    mlp.add(Dense(10, activation='softmax',input_dim=input_dim))
    # compile with standard sgd
    mlp.compile(optimizer=SGD(lr=eta), loss='binary_crossentropy', metrics='accuracy')
    # define tensorboard call-back
    callback_tensorboard = TensorBoard(log_dir = "./Illustration-tensorboard/mlp_sgd-eta-{}".format(eta), histogram_freq = 1)
    # fit
    mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), 
            callbacks = callback_tensorboard, batch_size=32, epochs=10, verbose=1) 


Epoch 1/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3418 - accuracy: 0.0681 - val_loss: 0.3409 - val_accuracy: 0.0685
Epoch 2/10
1500/1500 [==============================] - 1s 782us/step - loss: 0.3391 - accuracy: 0.0735 - val_loss: 0.3382 - val_accuracy: 0.0736
Epoch 3/10
1500/1500 [==============================] - 1s 784us/step - loss: 0.3364 - accuracy: 0.0791 - val_loss: 0.3356 - val_accuracy: 0.0796
Epoch 4/10
1500/1500 [==============================] - 1s 729us/step - loss: 0.3339 - accuracy: 0.0861 - val_loss: 0.3331 - val_accuracy: 0.0851
Epoch 5/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3314 - accuracy: 0.0938 - val_loss: 0.3307 - val_accuracy: 0.0935
Epoch 6/10
1500/1500 [==============================] - 1s 997us/step - loss: 0.3290 - accuracy: 0.1025 - val_loss: 0.3284 - val_accuracy: 0.1018
Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3267 - accuracy: 0.1117 - val_loss: 0.3261 - val_a

Epoch 7/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0470 - accuracy: 0.9230 - val_loss: 0.0474 - val_accuracy: 0.9210
Epoch 8/10
1500/1500 [==============================] - 1s 994us/step - loss: 0.0464 - accuracy: 0.9232 - val_loss: 0.0471 - val_accuracy: 0.9222
Epoch 9/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0460 - accuracy: 0.9250 - val_loss: 0.0469 - val_accuracy: 0.9223
Epoch 10/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.0455 - accuracy: 0.9258 - val_loss: 0.0474 - val_accuracy: 0.9204


## L'intérêt de tensorboard prend alors tout son sens : il est maintenant très facile d'explorer l'impact des hyper-paramètres sur l'apprentissage du modèle.

# 4) Analyse de l'impact de l'algorithme de descente de gradient 

## <span style="color:red"> EXERCICE : faire de même pour évaluer l'impact de l'algorithme d'optimisation à proprement parler, en considérant cette fois l'algorithme "Adam" </span>

Pour cela, il suffit d'importer et d'utiliser l'"optimizer" **Adam** comme nous nous l'avons fait ci-dessus pour le **SGD**.
Comme précédemment, nous ferons varier le paramètre de learning rate pour mesurer l'impact de la combinaison \[algorithme x learning-rate\]. 


In [10]:
##############################
# assess effect of optimizer #
##############################
learning_rates = [0.0001,0.001,0.01,0.1,0.5]
from tensorflow.keras.optimizers import Adam
for eta in learning_rates:
    print(eta)
    # create model
    mlp = Sequential()
    mlp.add(Dense(10, activation='softmax',input_dim=input_dim))

    # compile with adam
    mlp.compile(optimizer=Adam(learning_rate=eta), loss='binary_crossentropy', metrics='accuracy')

    # define tensorboard call-back
    callback_tensorboard = TensorBoard(log_dir = "./Illustration-tensorboard/mlp_adam-eta-{}".format(eta), histogram_freq = 1)
    # fit
    mlp.fit(X_train_vec_train, Y_train_train, validation_data = (X_train_vec_val,Y_train_val), 
            callbacks = callback_tensorboard, batch_size=32, epochs=10, verbose=1) 


0.0001
Epoch 1/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.1870 - accuracy: 0.7169 - val_loss: 0.1186 - val_accuracy: 0.8368
Epoch 2/10
1500/1500 [==============================] - 1s 915us/step - loss: 0.0983 - accuracy: 0.8594 - val_loss: 0.0850 - val_accuracy: 0.8721
Epoch 3/10
1500/1500 [==============================] - 1s 840us/step - loss: 0.0779 - accuracy: 0.8795 - val_loss: 0.0723 - val_accuracy: 0.8865
Epoch 4/10
1500/1500 [==============================] - 1s 830us/step - loss: 0.0688 - accuracy: 0.8906 - val_loss: 0.0656 - val_accuracy: 0.8943
Epoch 5/10
1500/1500 [==============================] - 1s 920us/step - loss: 0.0636 - accuracy: 0.8974 - val_loss: 0.0615 - val_accuracy: 0.9006
Epoch 6/10
1500/1500 [==============================] - 1s 847us/step - loss: 0.0603 - accuracy: 0.9017 - val_loss: 0.0586 - val_accuracy: 0.9045
Epoch 7/10
1500/1500 [==============================] - 1s 805us/step - loss: 0.0579 - accuracy: 0.9047 - val_loss: 0.0

Epoch 8/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6664 - accuracy: 0.7806 - val_loss: 0.6841 - val_accuracy: 0.7747
Epoch 9/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6452 - accuracy: 0.7876 - val_loss: 0.7578 - val_accuracy: 0.7509
Epoch 10/10
1500/1500 [==============================] - 2s 1ms/step - loss: 0.6265 - accuracy: 0.7940 - val_loss: 0.6271 - val_accuracy: 0.7937
